In [8]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [9]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)

## Do your thing crazy machine learning thing here :) ...

In [10]:
tX = correction_missing_values(tX)
tX, mean_x, std_x = standardize(tX)
tX = normalize(tX)
tX = np.c_[np.ones((y.shape[0], 1)), tX]

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

clf_logistic = LogisticRegression(random_state=0).fit(tX, y)
clf_SVC = SVC(gamma='auto').fit(tX,y)
print("Accuracy from scikit learn - logistic regression: "+str(round(100*clf_logistic.score(tX, y),5))+' %\n')
print("Accuracy from scikit learn - Support Vector Machine: "+str(round(100*clf_SVC.score(tX, y),5))+' %\n')

Accuracy from scikit learn - logistic regression: 73.94 %

Accuracy from scikit learn - Support Vector Machine: 67.44 %



In [12]:
from logistic_regression import *
#Using our code

gamma = 0.0001
n_iter = 10000

w = np.random.rand(tX.shape[1])
for i in range(n_iter):
    w, loss = learning_by_gradient_descent(y, tX, w, gamma)
    if i % 100 == 0:
        print("Current iteration={iter}, loss={l}".format(iter=i, l=loss))
print(w, loss)

print("Accuracy from our code:\n")
print("Logistic regression: "+str(round(100*np.sum(predict_labels(w, tX)==y)/len(y),5))+" %\n")

Current iteration=0, loss=14330.761585035349
Current iteration=100, loss=3101.08933151618
Current iteration=200, loss=3057.2066610554793
Current iteration=300, loss=3018.668010903647
Current iteration=400, loss=2984.665806532561
Current iteration=500, loss=2954.514071113862
Current iteration=600, loss=2927.636595649753
Current iteration=700, loss=2903.5524886894605
Current iteration=800, loss=2881.861575493349
Current iteration=900, loss=2862.2309572445456
Current iteration=1000, loss=2844.3832756247966
Current iteration=1100, loss=2828.0867879008238
Current iteration=1200, loss=2813.1471359345596
Current iteration=1300, loss=2799.4006006225713
Current iteration=1400, loss=2786.7086114545004
Current iteration=1500, loss=2774.953293163591
Current iteration=1600, loss=2764.033857751266
Current iteration=1700, loss=2753.8636800668914
Current iteration=1800, loss=2744.367923643686
Current iteration=1900, loss=2735.4816086012897
Current iteration=2000, loss=2727.148034588775
Current iterati

In [13]:
from reg_logistic_regression import *

gamma = 0.0001
lamda_ = 10
n_iter = 4000

w_reg = np.random.rand(tX.shape[1])
for i in range(n_iter):
    w_reg, loss = learning_by_penalized_gradient(y, tX, w_reg, gamma, lamda_)
    if i % 100 == 0:
        print("Current iteration={iter}, loss={l}".format(iter=i, l=loss))
print(w_reg, loss)

print("After regularization: "+str(round(100*np.sum(predict_labels(w_reg, tX)==y)/len(y),5))+" %\n")

Current iteration=0, loss=15546.767300777186
Current iteration=100, loss=3133.261547092958
Current iteration=200, loss=3106.307115632103
Current iteration=300, loss=3090.251614116
Current iteration=400, loss=3080.6315599854674
Current iteration=500, loss=3074.8327142736593
Current iteration=600, loss=3071.3165266504634
Current iteration=700, loss=3069.1722945593638
Current iteration=800, loss=3067.857632509518
Current iteration=900, loss=3067.0474982737655
Current iteration=1000, loss=3066.5459049651554
Current iteration=1100, loss=3066.2339808598604
Current iteration=1200, loss=3066.0392197311457
Current iteration=1300, loss=3065.9171603232758
Current iteration=1400, loss=3065.840402446046
Current iteration=1500, loss=3065.791981509788
Current iteration=1600, loss=3065.7613486835776
Current iteration=1700, loss=3065.741918395089
Current iteration=1800, loss=3065.7295642195013
Current iteration=1900, loss=3065.721691881364
Current iteration=2000, loss=3065.7166653215813
Current iterati

## Generate predictions and save ouput in csv format for submission:

In [6]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test = np.c_[np.ones((tX_test.shape[0], 1)), tX_test]

In [7]:
OUTPUT_PATH = 'test_reg_log_regr.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w_reg, tX_test)
y_pred[np.where(y_pred == 0)] = -1
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)